In [6]:
!pip install pandas numpy torch pytorch-lightning transformers scikit-learn sentencepiece tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from tqdm.auto import tqdm
import warnings

# --- Configuration ---
warnings.filterwarnings('ignore')
pl.seed_everything(42)

MODEL_NAME = 'google/flan-t5-xl'
BATCH_SIZE = 15
LEARNING_RATE = 3e-5
MAX_EPOCHS = 50
SOURCE_MAX_LEN = 256
TARGET_MAX_LEN = 8

# Price buckets - based on actual price distribution
# More granular in 0-500 range where most data is, wider for long tail
PRICE_BUCKETS = [0, 10, 25, 50, 100, 200, 350, 500, 1000, 3000]
NUM_BUCKETS = len(PRICE_BUCKETS) - 1
BUCKET_LABELS = [
    f"class{i+1}_{int(PRICE_BUCKETS[i])}_to_{int(PRICE_BUCKETS[i+1])}" 
    for i in range(NUM_BUCKETS)
]

# Print bucket information for clarity
print("\n📦 Price Buckets:")
for i, label in enumerate(BUCKET_LABELS):
    print(f"   {label}")

Seed set to 42



📦 Price Buckets:
   class1_0_to_10
   class2_10_to_25
   class3_25_to_50
   class4_50_to_100
   class5_100_to_200
   class6_200_to_350
   class7_350_to_500
   class8_500_to_1000
   class9_1000_to_3000


In [8]:
# --- SMAPE Metric ---
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    """Calculate SMAPE."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    denominator[denominator == 0] = 1e-8
    smape = np.mean(2 * np.abs(y_pred - y_true) / denominator) * 100
    return smape

def to_float(price_str):
    """Convert string to float."""
    try:
        return float(str(price_str).replace(',', ''))
    except (ValueError, TypeError):
        return 0.0

def price_to_bucket(price):
    """Map price to bucket index."""
    for i in range(NUM_BUCKETS):
        if price < PRICE_BUCKETS[i + 1]:
            return i
    return NUM_BUCKETS - 1

def bucket_to_label(bucket_idx):
    """Convert bucket index to class label."""
    return BUCKET_LABELS[bucket_idx]

def label_to_bucket(label):
    """Convert class label to bucket index."""
    try:
        return BUCKET_LABELS.index(label)
    except ValueError:
        return 0

def bucket_to_price(bucket_idx):
    """Convert bucket index to approximate price (midpoint of bucket range)."""
    lower = PRICE_BUCKETS[bucket_idx]
    upper = PRICE_BUCKETS[bucket_idx + 1]
    # Use geometric mean for better estimate across log-spaced buckets
    return np.sqrt(lower * upper) if lower > 0 else upper / 2

In [9]:
# --- Dataset ---
class T5MultiTaskDataset(Dataset):
    """Dataset with multi-task labels."""
    def __init__(self, dataframe, tokenizer, source_max_len, target_max_len, is_test=False):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_max_len = source_max_len
        self.target_max_len = target_max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source_text = str(self.data.iloc[index]['t5_input'])
        source = self.tokenizer.batch_encode_plus(
            [source_text], max_length=self.source_max_len,
            padding='max_length', truncation=True, return_tensors='pt'
        )
        
        result = {
            'input_ids': source['input_ids'].squeeze().to(dtype=torch.long),
            'attention_mask': source['attention_mask'].squeeze().to(dtype=torch.long)
        }

        if self.is_test:
            return result

        # For training: add bucket classification target
        price = float(self.data.iloc[index]['price'])
        bucket_idx = price_to_bucket(price)
        
        # Target for text generation (bucket class label)
        target_text = bucket_to_label(bucket_idx)
        target = self.tokenizer.batch_encode_plus(
            [target_text], max_length=self.target_max_len,
            padding='max_length', truncation=True, return_tensors='pt'
        )
        
        result.update({
            'labels': target['input_ids'].squeeze().to(dtype=torch.long),
            'price': torch.tensor(price, dtype=torch.float32),
            'bucket_idx': torch.tensor(bucket_idx, dtype=torch.long)
        })
        
        return result

# --- Multi-Task Model ---
class T5MultiTaskPredictor(pl.LightningModule):
    """T5 with multi-task learning: bucket class generation + bucket classification."""
    def __init__(self, model_name, learning_rate, tokenizer, train_dataset_len, batch_size, max_epochs):
        super().__init__()
        self.save_hyperparameters(ignore=['tokenizer'])
        
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.tokenizer = tokenizer
        
        # Add classification head on top of encoder
        encoder_dim = self.model.config.d_model
        self.bucket_classifier = nn.Sequential(
            nn.Linear(encoder_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, NUM_BUCKETS)
        )
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.validation_step_outputs = []

    def forward(self, input_ids, attention_mask, labels=None):
        # Get encoder hidden states
        encoder_outputs = self.model.encoder(input_ids=input_ids, attention_mask=attention_mask)
        encoder_hidden = encoder_outputs.last_hidden_state
        
        # Main T5 generation loss (for bucket class labels)
        outputs = self.model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            labels=labels,
            encoder_outputs=encoder_outputs
        )
        
        # Bucket classification from encoder
        # Use mean pooling
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(encoder_hidden.size()).float()
        sum_hidden = torch.sum(encoder_hidden * attention_mask_expanded, dim=1)
        sum_mask = attention_mask_expanded.sum(dim=1).clamp(min=1e-9)
        pooled = sum_hidden / sum_mask
        
        bucket_logits = self.bucket_classifier(pooled)
        
        return outputs.loss, outputs.logits, bucket_logits

    def training_step(self, batch, batch_idx):
        gen_loss, _, bucket_logits = self(
            batch['input_ids'], batch['attention_mask'], batch['labels']
        )
        
        # Auxiliary bucket classification loss
        bucket_loss = self.ce_loss(bucket_logits, batch['bucket_idx'])
        
        # Combined loss: 70% generation, 30% bucket classification
        total_loss = 0.7 * gen_loss + 0.3 * bucket_loss
        
        self.log('train_loss', total_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_gen_loss', gen_loss, on_epoch=True)
        self.log('train_bucket_loss', bucket_loss, on_epoch=True)
        
        return total_loss

    def validation_step(self, batch, batch_idx):
        gen_loss, _, bucket_logits = self(
            batch['input_ids'], batch['attention_mask'], batch['labels']
        )
        
        bucket_loss = self.ce_loss(bucket_logits, batch['bucket_idx'])
        total_loss = 0.7 * gen_loss + 0.3 * bucket_loss
        
        self.log('val_loss', total_loss, on_epoch=True, prog_bar=True)
        
        # Generate predictions
        generated_ids = self.model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=TARGET_MAX_LEN,
            num_beams=3,
            early_stopping=True
        )
        
        preds = [self.tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
        
        # Convert class predictions to actual prices
        prices_pred = []
        for pred in preds:
            try:
                bucket_idx = label_to_bucket(pred)
            except:
                bucket_idx = 0
            price = bucket_to_price(bucket_idx)
            prices_pred.append(price)
        
        self.validation_step_outputs.append({
            'preds': np.array(prices_pred),
            'targets': batch['price'].cpu().numpy()
        })
        
        return total_loss

    def on_validation_epoch_end(self):
        all_preds = np.concatenate([out['preds'] for out in self.validation_step_outputs])
        all_targets = np.concatenate([out['targets'] for out in self.validation_step_outputs])
        
        all_preds = np.clip(all_preds, 0, None)
        
        val_smape = symmetric_mean_absolute_percentage_error(all_targets, all_preds)
        self.log('val_smape', val_smape, on_epoch=True, prog_bar=True, logger=True)
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=0.01)
        num_training_steps = (self.hparams.train_dataset_len // self.hparams.batch_size) * self.hparams.max_epochs
        num_warmup_steps = int(num_training_steps * 0.05)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

In [5]:
print("=" * 80)
print("APPROACH 2: Log-Transform + Multi-Task Learning")
print("=" * 80)

# 1. Load Data
train_df = pd.read_csv('/root/train.csv', encoding='latin1')
test_df = pd.read_csv('/root/test.csv', encoding='latin1')

# 2. Preprocess
print("\n📝 Applying enhanced text cleaning...")

train_df['catalog_content'] = train_df['catalog_content'].astype(str)
test_df['catalog_content'] = test_df['catalog_content'].astype(str)

train_df['t5_input'] = "predict price: " + train_df['catalog_content']
train_df['t5_target'] = train_df['price'].astype(str)
test_df['t5_input'] = "predict price: " + test_df['catalog_content']


# 3. Split
train_split_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)
print(f"📊 Training: {len(train_split_df)}, Validation: {len(val_df)}")

# 4. Initialize
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
train_dataset = T5MultiTaskDataset(train_split_df, tokenizer, SOURCE_MAX_LEN, TARGET_MAX_LEN)
val_dataset = T5MultiTaskDataset(val_df, tokenizer, SOURCE_MAX_LEN, TARGET_MAX_LEN)

# 5. DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# 6. Model
model = T5MultiTaskPredictor(
    model_name=MODEL_NAME, learning_rate=LEARNING_RATE, tokenizer=tokenizer,
    train_dataset_len=len(train_dataset), batch_size=BATCH_SIZE, max_epochs=MAX_EPOCHS
)

checkpoint_callback = ModelCheckpoint(
    dirpath='/mnt/approach2-log-multitask/checkpoints',
    filename='best-log-multitask',
    save_top_k=1, verbose=True, monitor='val_smape', mode='min'
)
early_stopping_callback = EarlyStopping(monitor='val_smape', patience=10, mode='min')

trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=MAX_EPOCHS, accelerator='gpu', devices=1, precision='bf16-mixed'
)

# 7. Train
print("\n🚀 Training with Log-Transform + Multi-Task Learning...")
trainer.fit(model, train_loader, val_loader)

# 8. Inference
print("\n🔮 Starting inference...")
best_model = T5MultiTaskPredictor.load_from_checkpoint(
    checkpoint_callback.best_model_path, tokenizer=tokenizer
)
best_model.freeze()
best_model.eval()
best_model.to('cuda' if torch.cuda.is_available() else 'cpu')

test_dataset = T5MultiTaskDataset(test_df, tokenizer, SOURCE_MAX_LEN, TARGET_MAX_LEN, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=8)

predictions = []
for batch in tqdm(test_loader, desc="Predicting"):
    batch = {k: v.to(best_model.device) for k, v in batch.items()}
    with torch.no_grad():
        generated_ids = best_model.model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=TARGET_MAX_LEN,
            num_beams=5,
            early_stopping=True
        )
        preds = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]
        
        # Convert class predictions to prices
        for pred in preds:
            try:
                bucket_idx = label_to_bucket(pred)
            except:
                bucket_idx = 0
            price = bucket_to_price(bucket_idx)
            predictions.append(price)

# 9. Submission
test_df['price'] = np.array(predictions).clip(min=0)
submission_df = test_df[['sample_id', 'price']]
submission_df.to_csv('/mnt/approach2-log-multitask/submission_approach2.csv', index=False)

print("\n✅ Approach 2 Complete! Submission saved.")
print(submission_df.head())
print(f"\n📈 Price Statistics:")
print(f"   Min: ${submission_df['price'].min():.2f}")
print(f"   Max: ${submission_df['price'].max():.2f}")
print(f"   Mean: ${submission_df['price'].mean():.2f}")
print(f"   Median: ${submission_df['price'].median():.2f}")

SystemExit: 1

APPROACH 2: Log-Transform + Multi-Task Learning

📝 Applying enhanced text cleaning...
📊 Training: 63750, Validation: 11250


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



🚀 Training with Log-Transform + Multi-Task Learning...



  | Name              | Type                       | Params | Mode 
-------------------------------------------------------------------------
0 | model             | T5ForConditionalGeneration | 2.8 B  | eval 
1 | bucket_classifier | Sequential                 | 526 K  | train
2 | ce_loss           | CrossEntropyLoss           | 0      | train
-------------------------------------------------------------------------
2.9 B     Trainable params
0         Non-trainable params
2.9 B     Total params
11,401.136Total estimated model params size (MB)
6         Modules in train mode
1117      Modules in eval mode


Sanity Checking: |                                                                       | 0/? [00:00<?, ?it/s…

Training: |                                                                              | 0/? [00:00<?, ?it/s…